In [1]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import cv2
from evaluator import ModelEvaluator
from tqdm import tqdm

c:\Users\PARK\anaconda3\envs\eai\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Device 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [3]:
# 데이터셋 경로
data_dir = "data"

In [4]:
# 전처리
transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [ ]:
# 데이터셋 로드
dataset = ImageFolder(root=data_dir, transform=transform)
# dataset = ImageFolder(root=data_dir)

# 클래스 정보 출력
print(f"Classes: {dataset.classes}")

Classes: ['with_mask', 'without_mask']


In [6]:
# Train:Val:Test = 70:15:15 분할
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size
train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])


In [7]:
# DataLoader 생성
batch_size = 32
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [8]:
class MaskClassifier(nn.Module):
    def __init__(self):
        super(MaskClassifier, self).__init__()
        
        # Feature Extraction - 더 얕은 구조로 변경
        self.features = nn.Sequential(
            # First Block
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.2),
            
            # Second Block
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.2),
            
            # Third Block
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            nn.Dropout2d(0.2),
        )
        
        # Classifier - 더 단순한 구조로 변경
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [9]:
model = MaskClassifier().to(device)

In [10]:
# criterion = nn.CrossEntropyLoss()
# Loss function 수정 - 클래스 불균형을 고려한 가중치 추가
weights = torch.tensor([1.0, 1.0]).to(device)  # 필요시 클래스별 가중치 조정
criterion = nn.CrossEntropyLoss(weight=weights)

# Learning rate와 optimizer 수정
learning_rate = 0.001  # 더 작은 learning rate 사용
num_epoch = 50  # epoch 수 감소

optimizer = optim.Adam(
    model.parameters(),
    lr=learning_rate,
    betas=(0.9, 0.999),
    weight_decay=0.0001  # 더 작은 weight decay
)

In [11]:
# 학습 함수
def train_model(model, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}", unit="batch")  # Progress Bar
        
        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

            # Progress Bar에 현재 배치 손실 표시
            progress_bar.set_postfix(loss=loss.item())

        val_loss = 0
        model.eval()
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

In [12]:
# 학습 실행
train_model(model, train_loader, val_loader, num_epoch)

Epoch 1/50:   0%|          | 0/112 [00:00<?, ?batch/s]


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.Image.Image'>

In [ ]:
# 테스트 함수
def test_model(model, test_loader):
    """
    Args:
        model (torch.nn.Module): 평가할 모델
        test_loader (DataLoader): 테스트 데이터 로더
    """
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        progress_bar = tqdm(test_loader, desc="Testing", unit="batch")  # Progress Bar 추가
        for images, labels in progress_bar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

            # Progress Bar 상태 업데이트 (현재 배치의 예측 결과 일부 표시)
            progress_bar.set_postfix(batch_accuracy=(preds == labels).float().mean().item())

    print("\nTest Classification Report:")
    print(classification_report(all_labels, all_preds, target_names=dataset.classes))


In [ ]:
# 테스트 실행
test_model(model, test_loader)

Testing: 100%|██████████| 9/9 [00:03<00:00,  2.93batch/s, batch_accuracy=0.857]


Test Classification Report:
              precision    recall  f1-score   support

   with_mask       0.97      0.93      0.95       152
without_mask       0.91      0.95      0.93       111

    accuracy                           0.94       263
   macro avg       0.94      0.94      0.94       263
weighted avg       0.94      0.94      0.94       263



In [ ]:
# 웹캠 테스트 함수
def webcam_test(model):
    model.eval()
    cap = cv2.VideoCapture(0)  # 웹캠 열기
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((112, 112)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5])
    ])

    print("Press 'q' to quit.")
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        input_tensor = transform(image).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(input_tensor)
            _, pred = torch.max(output, 1)

        label = dataset.classes[pred.item()]
        cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow("Webcam", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [ ]:
# 웹캠 테스트 실행
webcam_test(model)

Press 'q' to quit.


In [ ]:
# 모델 저장 코드
def save_model(model, path="mask_classifier.pth"):
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")

In [ ]:
# 학습 후 모델 저장 및 평가
save_model(model, "mask_classifier.pth")

# 모델 정의 및 학습 완료 후
evaluator = ModelEvaluator(model, device=device)
evaluator.evaluate_all(input_size=(3, 112, 112), iterations=100)